In [26]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential
from PIL import Image
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
# print(pd.__version__)
import numpy as np

In [2]:
TEST = 'data\\test'
TRAIN = 'data\\train'

In [3]:
def dataframe(dir):
    image_paths = []
    labels = []
    # print( os.listdir(dir))
    for label in os.listdir(dir):
        # print(label)
        path = os.path.join(dir , label)
        for image_name in os.listdir(path):
            # print(image_name)
            image_paths.append(os.path.join(path ,image_name))
            labels.append(label)
    return image_paths , labels

In [4]:
# dataframe(TEST)
train = pd.DataFrame()
train['image'] , train['label'] = dataframe(TRAIN)
# print(train['image'])
# print(train['label'])

In [5]:
teset = pd.DataFrame()
teset['image'] , teset['label'] = dataframe(TEST)
# print(teset['image'])
# print(teset['label'])

In [6]:
from tqdm.notebook import tqdm
from tqdm import tqdm
def features_exact(images):
    features = []
    for image in tqdm(images):
        # print(image)
        img = load_img(image, color_mode='grayscale')  
        img = np.array(img)  
        features.append(img)  

    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1) 
    return features

In [11]:
train_features = features_exact(train['image'])
# print(train_features)

100%|██████████| 28709/28709 [01:23<00:00, 343.36it/s] 


In [8]:
test_features = features_exact(teset['image'])

100%|██████████| 7178/7178 [01:05<00:00, 109.38it/s]


In [9]:
x_train = train_features/255.0
x_test = test_features/255.0

In [10]:
from sklearn.preprocessing import LabelEncoder

In [19]:
lebal = LabelEncoder()
lebal.fit(train['label'])

LabelEncoder()

In [20]:
y_train = lebal.transform(train['label'])

In [21]:
print(teset['label'])
y_test = lebal.transform(teset['label'])

0           angry
1           angry
2           angry
3           angry
4           angry
          ...    
7173    surprised
7174    surprised
7175    surprised
7176    surprised
7177    surprised
Name: label, Length: 7178, dtype: object


In [22]:
y_train = to_categorical(y_train ,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [27]:
emotion_model = Sequential()

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.4))  # Use Dropout instead of Dense

emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.4))  # Use Dropout instead of Dense

emotion_model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.4))  # Use Dropout instead of Dense

emotion_model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.4))  # Use Dropout instead of Dense

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(512, activation='relu'))
emotion_model.add(Dropout(0.5))

emotion_model.add(Dense(7, activation='softmax'))

# Learning rate schedule and optimizer
initial_learning_rate = 0.0001
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=100000, decay_rate=0.96)
optimizer = Adam(learning_rate=lr_schedule)
emotion_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
emotion_model.fit(x= x_train , y= y_train , batch_size = 128 , epochs = 50 , validation_data = (x_test ,y_test))

Epoch 1/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 206s 913ms/step - accuracy: 0.3990 - loss: 1.5202 - val_accuracy: 0.4472 - val_loss: 1.4379
Epoch 2/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 205s 912ms/step - accuracy: 0.4388 - loss: 1.4509 - val_accuracy: 0.4799 - val_loss: 1.3732
Epoch 3/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 207s 920ms/step - accuracy: 0.4579 - loss: 1.4010 - val_accuracy: 0.4985 - val_loss: 1.3231
Epoch 4/50
225/225 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 0.4842 - loss: 1.3591 - val_accuracy: 0.5067 - val_loss: 1.2931
Epoch 5/50
 28/225 ━━━━━━━━━━━━━━━━━━━━ 3:33 1s/step - accuracy: 0.5122 - loss: 1.3036   

In [ ]:
model_json = emotion_model.to_json()
with open("emotiondetectore.json" , "w") as json_file:
    json_file.write(model_json)
emotion_model.save("emotiondetectore.h5")